In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ETL")\
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.6.2') \
        .getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

In [6]:
product_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://103.245.236.247:5432/e-commerce") \
    .option("driver", "org.postgresql.Driver")\
    .option("user", "traianthai") \
    .option("password", "123123") \
    .option("dbtable", "public.\"product\"") \
    .load()


In [7]:
product_df.show()

+---+--------------------+--------------------+---------+--------------------+-----+--------------------+--------------------+----------+-------+
| id|           createdAt|           updatedAt|deletedAt|                name|price|         description|                 img|categoryId|brandId|
+---+--------------------+--------------------+---------+--------------------+-----+--------------------+--------------------+----------+-------+
|301|2024-04-26 10:49:...|2024-04-26 10:49:...|     NULL|[CHỈ GIAO HCM] Hà...|17000|Hành tây tím có r...|https://salt.tiki...|        15|      4|
|104|2024-04-26 10:49:...|2024-04-26 10:49:...|     NULL|    Cải Thảo (800gr)|50000|Cải thảo là một l...|https://salt.tiki...|         1|      6|
|174|2024-04-26 10:49:...|2024-04-26 10:49:...|     NULL|Cải Bẹ Xanh Thủy ...|31000|Cải bẹ xanh (còn ...|https://salt.tiki...|         1|      6|
|259|2024-04-26 10:49:...|2024-04-26 10:49:...|     NULL|Cà chua beef Viet...|22000|       Cà chua be...|https://salt.tiki..

In [9]:
from pyspark.sql.functions import col

# Tạo DataFrame mới chỉ chứa cột "id" và "name"
product_df = product_df.select(col("id"), col("name"), col("price"), col("brandId"), col("categoryId"))

# Hiển thị DataFrame mới
product_df.show()


+---+--------------------+-----+-------+----------+
| id|                name|price|brandId|categoryId|
+---+--------------------+-----+-------+----------+
|301|[CHỈ GIAO HCM] Hà...|17000|      4|        15|
|104|    Cải Thảo (800gr)|50000|      6|         1|
|174|Cải Bẹ Xanh Thủy ...|31000|      6|         1|
|259|Cà chua beef Viet...|22000|      1|         3|
| 45|[Chỉ Giao HCM] - ...|30000|      2|         3|
|144|Khoai Môn Gọt Vỏ ...|45000|      6|        13|
|160|Nấm Bào Ngư Xám T...|22000|      6|         2|
|262|[CHỈ GIAO HCM] Cả...| 9000|      1|         1|
|271|   Ngò Om (Gói 50Gr)| 8000|      6|         1|
|258|Rong biển khô nấu...|30000|      1|         3|
|108|Măng Tây (Gói 200Gr)|45000|      6|         1|
|273|       Bắp Mỹ (Trái)|19000|      6|         1|
| 28|Giá Sạch (Gói 300Gr)|15000|      6|         3|
|  2|Cải Bó Xôi (Gói 2...|37000|      6|         1|
| 29|[Chỉ Giao HCM] - ...|20000|      2|        12|
|198|      Khoai Mỡ - 1Kg|85000|      6|        13|
|251|Đậu Hà 

In [10]:
# Ghi DataFrame vào bảng "Dim_Product" trong schema "public"
product_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://103.245.236.247:5432/data_warehouse") \
    .option("dbtable", "public.\"Dim_Product\"") \
    .option("user", "traianthai") \
    .option("password", "123123") \
    .mode("overwrite") \
    .save()


In [11]:
# Đóng SparkSession
spark.stop()